importing

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
df_ords = pd.read_csv(r'C:\Users\bodhi\Downloads\instacart basket analysis\data\original data\orders.csv', index_col = False)
df_prods = pd.read_csv(r'C:\Users\bodhi\Downloads\instacart basket analysis\data\original data\products.csv', index_col = False)

In [3]:
#create a dataframe
df_test = pd.DataFrame()

In [4]:
#create a mixed type column
df_test['mix'] = [ 'a', 'b', 1, True]

In [5]:
#convert data types to string
df_test['mix'] = df_test['mix'].astype('str')

consistency checks

In [6]:
df_prods.isnull().sum()

product_id        0
product_name     16
aisle_id          0
department_id     0
prices            0
dtype: int64

In [7]:
df_nan = df_prods[df_prods['product_name'].isnull() == True]

In [8]:
df_prods.shape

(49693, 5)

In [9]:
df_prods_clean = df_prods[df_prods['product_name'].isnull() == False]

In [10]:
df_prods_clean.shape

(49677, 5)

In [11]:
df_dups = df_prods_clean[df_prods_clean.duplicated()]

In [12]:
df_prods_clean_no_dups = df_prods_clean.drop_duplicates()

In [13]:
#exporting
df_prods_clean_no_dups.to_csv(r"C:\Users\bodhi\Downloads\instacart basket analysis\data\prepared data\products_checked.csv")

In [14]:
#Set pandas display option to disable scientific notation
pd.set_option('display.float_format', '{:.2f}'.format)


task

In [15]:
df_ords.describe()

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order
count,3421083.00,3421083.00,3421083.00,3421083.00,3421083.00,3214874.00
mean,1710542.00,102978.21,17.15,2.78,13.45,11.11
std,987581.74,59533.72,17.73,2.05,4.23,9.21
min,1.00,1.00,1.00,0.00,0.00,0.00
25%,855271.50,51394.00,5.00,1.00,10.00,4.00
50%,1710542.00,102689.00,11.00,3.00,13.00,7.00
75%,2565812.50,154385.00,23.00,5.00,16.00,15.00
max,3421083.00,206209.00,100.00,6.00,23.00,30.00


order day or week max is 6, so is the shop closed one day a week?
order hour of day max is 23, so is it closed one hour per day?

step 3

In [19]:
# Iterate through all columns in the DataFrame
for col in df_ords.columns.tolist():
    # Check for type inconsistencies in each column using map instead of applymap
    col_types = df_ords[[col]].map(type)
    first_type = df_ords[col].iloc[0].__class__
    weird = (col_types != first_type).any(axis=1)
    
    if len(df_ords[weird]) > 0:
        print(col)  # Print column name with inconsistent types

order_id
user_id
order_number
order_dow
order_hour_of_day
days_since_prior_order


In [32]:
columns_to_convert = ['user_id', 'order_id', 'order_number', 'order_dow', 'order_hour_of_day', 'days_since_prior_order']

for col in columns_to_convert:
    df_ords[col] = df_ords[col].astype('float')



In [33]:
df_ords.isnull().sum()

order_id                       0
user_id                        0
eval_set                       0
order_number                   0
order_dow                      0
order_hour_of_day              0
days_since_prior_order    206209
dtype: int64

there are a lot of missing values in the days since prior order column. this is most likely because it was that customers first order. the best solution would be to create a new column that says either first order, or not.

In [34]:
df_ords['order_type'] = df_ords['days_since_prior_order'].isna().map({True: 'First Order', False: 'Repeat Order'})

In [35]:
df_ords

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,order_type
0,2539329.00,1.00,prior,1.00,2.00,8.00,NaN,First Order
1,2398795.00,1.00,prior,2.00,3.00,7.00,15.00,Repeat Order
2,473747.00,1.00,prior,3.00,3.00,12.00,21.00,Repeat Order
3,2254736.00,1.00,prior,4.00,4.00,7.00,29.00,Repeat Order
4,431534.00,1.00,prior,5.00,4.00,15.00,28.00,Repeat Order
...,...,...,...,...,...,...,...,...
3421078,2266710.00,206209.00,prior,10.00,5.00,18.00,29.00,Repeat Order
3421079,1854736.00,206209.00,prior,11.00,4.00,10.00,30.00,Repeat Order
3421080,626363.00,206209.00,prior,12.00,1.00,12.00,18.00,Repeat Order
3421081,2977660.00,206209.00,prior,13.00,1.00,12.00,7.00,Repeat Order


In [36]:
df_ords_dups=df_ords[df_ords.duplicated()]

In [37]:
df_ords_dups

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,order_type


In [38]:
#exporting
df_ords.to_csv(r"C:\Users\bodhi\Downloads\instacart basket analysis\data\prepared data\orders_checked.csv")